# <a id='model-selection-and-evaluation'></a> 3. [**Sélection et évaluation de modèle**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#model-selection-and-evaluation)</br>([*Model selection and evaluation*](https://scikit-learn.org/stable/model_selection.html#model-selection-and-evaluation))

# 3.2. [**Réglage des hyper-paramètres d'un estimateur**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#tuning-the-hyper-parameters-of-an-estimator)<br/>([_Tuning the hyper-parameters of an estimator_](https://scikit-learn.org/stable/model_selection.html#tuning-the-hyper-parameters-of-an-estimator))

# Sommaire

- **Volume** : 18 pages, 9 exemples, 3 papiers
- 3.2.1. [**Recherche exhaustive en grille**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#exhaustive-grid-search)<br/>([_Exhaustive Grid Search_](https://scikit-learn.org/stable/model_selection.html#exhaustive-grid-search))
- 3.2.2. [**Optimisation aléatoire des paramètres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#randomized-parameter-optimization)<br/>([_Randomized Parameter Optimization_](https://scikit-learn.org/stable/model_selection.html#randomized-parameter-optimization))
- 3.2.3. [**Recherche de paramètres optimaux par halving successifs**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#searching-for-optimal-parameters-with-successive-halving)<br/>([_Searching for optimal parameters with successive halving_](https://scikit-learn.org/stable/model_selection.html#searching-for-optimal-parameters-with-successive-halving))
- 3.2.4. [**Conseils pour la recherche de paramètres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#tips-for-parameter-search)<br/>([_Tips for parameter search_](https://scikit-learn.org/stable/model_selection.html#tips-for-parameter-search))
- 3.2.5. [**Alternatives à la recherche de paramètres par force brute**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#alternatives-to-brute-force-parameter-search)<br/>([_Alternatives to brute force parameter search_](https://scikit-learn.org/stable/model_selection.html#alternatives-to-brute-force-parameter-search))

Les hyperparamètres sont des paramètres qui ne sont pas directement appris au sein des estimateurs. Dans scikit-learn, ils sont transmis en tant qu'arguments au constructeur des classes d'estimateur. Des exemples typiques incluent `C`, `kernel` et `gamma` pour le classifieur à vecteurs de support, `alpha` pour Lasso, etc.

Il est possible et recommandé d'explorer l'espace des hyperparamètres pour obtenir le meilleur score de [**validation croisée** (3.1)](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation).

Tout paramètre fourni lors de la construction d'un estimateur peut être optimisé de cette manière. Pour trouver les noms et les valeurs actuelles de tous les paramètres pour un estimateur donné, utilisez :

```python
estimator.get_params()
```

Une recherche comprend :
- un estimateur (régresseur ou classifieur tel que `sklearn.svm.SVC()`);
- un espace de paramètres ;
- une méthode pour rechercher ou échantillonner des candidats ;
- un schéma de validation croisée ; et
- une [**fonction de score** (3.2.4.1)](https://scikit-learn.org/stable/modules/grid_search.html#gridsearch-scoring).

Deux approches génériques de recherche de paramètres sont fournies dans scikit-learn : pour des valeurs données, [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) considère exhaustivement toutes les combinaisons de paramètres, tandis que [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) peut échantillonner un nombre donné de candidats dans un espace de paramètres avec une distribution spécifiée. Ces deux outils ont des contreparties successives, [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) et [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html), qui peuvent être beaucoup plus rapides pour trouver une bonne combinaison de paramètres.

Après avoir décrit ces outils, nous détaillons les [**meilleures pratiques** (3.2.4)](https://scikit-learn.org/stable/modules/grid_search.html#grid-search-tips) applicables à ces approches. Certains modèles permettent des stratégies de recherche de paramètres spécialisées et efficaces, décrites dans [**Alternatives to brute force parameter search** (3.2.5)](https://scikit-learn.org/stable/modules/grid_search.html#alternative-cv).

Notez qu'il est courant qu'un petit sous-ensemble de ces paramètres puisse avoir un impact important sur les performances prédictives ou calculatoires du modèle, tandis que d'autres peuvent être laissés à leurs valeurs par défaut. Il est recommandé de lire la documentation de la classe d'estimateur pour mieux comprendre leur comportement attendu, éventuellement en lisant la référence contenue dans la documentation.

## <a id='exhaustive-grid-search'></a> 3.2.1. Recherche exhaustive en grille

La recherche en grille fournie par [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) génère de manière exhaustive des candidats à partir d'une grille de valeurs de paramètres spécifiée avec le paramètre `param_grid`. Par exemple, la grille suivante :

In [1]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]

L'instance [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) implémente l'API habituelle de l'estimateur : lorsqu'elle est "ajustée" sur un ensemble de données, toutes les combinaisons possibles de valeurs de paramètres sont évaluées et la meilleure combinaison est conservée.

### Exemples

#### [**Stratégie personnalisée de réajustement d'une recherche en grille avec validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_digits.ipynb)<br/>([_Custom refit strategy of a grid search with cross-validation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html))

Un exemple de calcul de recherche en grille sur l'ensemble de données "digits".

#### [**Exemple de pipeline pour l'extraction et l'évaluation de caractéristiques de texte**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_text_feature_extraction.ipynb)<br/>([_Sample pipeline for text feature extraction and evaluation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_text_feature_extraction.html))
Un exemple d'utilisation de la recherche en grille couplant des paramètres d'un extracteur de caractéristiques de documents texte (comptage des n-grammes et transformation TF-IDF) avec un classifieur (ici un SVM linéaire entraîné avec SGD avec soit une régularisation de type elastic net, soit une régularisation de type L2) à l'aide d'une instance `pipeline.Pipeline`.

#### [**Validation croisée imbriquée vs. non-imbriquée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_nested_cross_validation_iris.ipynb)<br/>([_Nested versus non-nested cross-validation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html))

Un exemple de recherche en grille dans une boucle de validation croisée sur l'ensemble de données "Iris". Il s'agit de la meilleure pratique pour évaluer les performances d'un modèle avec la recherche en grille.

#### [**Démonstration de l'évaluation multi-métrique sur `cross_val_score` et `GridSearchCV`**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_multi_metric_evaluation.ipynb)<br/>([_Demonstration of multi-metric evaluation on `cross_val_score` and `GridSearchCV`_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html))

Un exemple d'utilisation de [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) pour évaluer simultanément plusieurs métriques.

#### [**Équilibrer la complexité du modèle et le score validé par validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_refit_callable.ipynb)<br/>([_Balance model complexity and cross-validated score_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_refit_callable.html))

Un exemple d'utilisation de l'interface `refit=callable` dans [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). L'exemple montre comment cette interface ajoute une certaine flexibilité pour identifier le meilleur estimateur. Cette interface peut également être utilisée dans l'évaluation de plusieurs métriques.

#### [**Comparaison statistique des modèles à l'aide de la recherche en grille**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_stats.ipynb)<br/>([_Statistical comparison of models using grid search_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_stats.html))

Un exemple de comparaison statistique des résultats de [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).